In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
import numpy as np
from tqdm import tqdm

In [ ]:
class TimeEmbedding(tf.keras.layers.Layer):
    def __init__(self, dim):
        super(TimeEmbedding, self).__init__()
        self.dim = dim
        self.half_dim = dim // 2
        self.emb = tf.math.log(10000.0) / (self.half_dim - 1)
        self.emb = tf.exp(tf.range(self.half_dim, dtype=tf.float32) * -self.emb)

    def call(self, timesteps):
        timesteps = tf.cast(timesteps, tf.float32)
        emb = timesteps[:, None] * self.emb[None, :]
        emb = tf.concat([tf.sin(emb), tf.cos(emb)], axis=-1)
        return emb

In [ ]:
class ResNetBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size):
        super(ResNetBlock, self).__init__()
        self.conv2d = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.group_norm = tf.keras.layers.GroupNormalization(axis=-1)
        self.gelu = tf.keras.layers.Activation('gelu')
        self.conv2d_2 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.group_norm_2 = tf.keras.layers.GroupNormalization(axis=-1)
        self.adjust_dim = tf.keras.layers.Conv2D(filters, 1, padding='same')

    def call(self, inputs):
        if tf.rank(inputs) == 3:
          inputs = tf.expand_dims(inputs, axis=1)
        x = self.conv2d(inputs)
        x = self.group_norm(x)
        x = self.gelu(x)
        x = self.conv2d_2(x)
        x = self.group_norm_2(x)

        inputs = self.adjust_dim(inputs)

        return tf.keras.layers.Add()([inputs, x])

In [ ]:
class DownSampleBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size):
        super(DownSampleBlock, self).__init__()
        self.max_pool = tf.keras.layers.MaxPool2D((2, 2))
        self.resnet_block = ResNetBlock(filters, kernel_size)
        self.resnet_block_2 = ResNetBlock(filters, kernel_size)
        self.linear = tf.keras.layers.Dense(filters)

    def call(self, inputs):
        x, y = inputs
        x = self.max_pool(x)
        x = self.resnet_block(x)
        x = self.resnet_block_2(x)

        y = tf.keras.activations.silu(y)
        y = self.linear(y)
        y = tf.reshape(y, [y.shape[0], 1, 1, y.shape[-1]])

        return x + y

In [ ]:
class SelfAttentionBlock(tf.keras.layers.Layer):
    def __init__(self, filters, num_heads=8, embed_dim=64):
        super(SelfAttentionBlock, self).__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.linear1 = tf.keras.layers.Dense(filters)
        self.gelu = tf.keras.layers.Activation('gelu')
        self.linear2 = tf.keras.layers.Dense(filters)

    def call(self, inputs):
        x = tf.reshape(inputs, shape=(inputs.shape[0], -1, inputs.shape[-1]))
        x_norm = self.layernorm1(x)
        x_mha = self.mha(query=x_norm, key=x_norm, value=x_norm)
        x_mha_out = x_mha + x
        x_mha_out = self.layernorm2(x_mha_out)
        x_ffn = self.linear1(x_mha_out)
        x_ffn = self.gelu(x_ffn)
        x_ffn = self.linear2(x_ffn)
        return x_ffn + x_mha_out

In [ ]:
class UpsampleBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size):
        super(UpsampleBlock, self).__init__()
        self.upsample = tf.keras.layers.UpSampling2D(size=(2, 2), interpolation='nearest')
        self.resnet_block = ResNetBlock(filters, kernel_size)
        self.resnet_block_2 = ResNetBlock(filters, kernel_size)
        self.linear = tf.keras.layers.Dense(filters)
        self.reduce_skip = tf.keras.layers.Conv2D(filters, 1, padding='same')

    def call(self, inputs):
        x, skip, t = inputs

        x = self.upsample(x)

        skip = self.reduce_skip(skip)

        if skip.shape[1] != x.shape[1] or skip.shape[2] != x.shape[2]:
            skip = tf.image.resize(skip, [x.shape[1], x.shape[2]])

        x = tf.image.resize(x, [skip.shape[1], skip.shape[2]])

        x_concat = tf.concat([x, skip], axis=-1)

        x_resnet_1 = self.resnet_block(x_concat)
        x_resnet_2 = self.resnet_block_2(x_resnet_1)

        t = tf.keras.activations.silu(t)
        t = self.linear(t)

        t = tf.expand_dims(t, axis=1)
        t = tf.expand_dims(t, axis=1)

        t = tf.tile(t, [1, 8, 8, 1])

        x_resnet_2 = tf.image.resize(x_resnet_2, [t.shape[1], t.shape[2]])
        return x_resnet_2 + t

In [ ]:
class DiffusionModel(tf.keras.Model):
    def __init__(self, img_size=32, base_filters=64, time_embedding_dim=256):
        super(DiffusionModel, self).__init__()
        self.time_embedding = TimeEmbedding(time_embedding_dim)
        self.first_resnet = ResNetBlock(base_filters * 8, 3)
        self.init_conv = tf.keras.layers.Conv2D(base_filters, 3, padding='same')

        self.down1 = DownSampleBlock(base_filters * 2, 3)
        self.down2 = DownSampleBlock(base_filters * 4, 3)
        self.down3 = DownSampleBlock(base_filters * 8, 3)

        self.mid_resnet1 = ResNetBlock(base_filters * 8, 3)
        self.mid_attention = SelfAttentionBlock(base_filters * 8)
        self.mid_resnet2 = ResNetBlock(base_filters * 8, 3)

        self.up1 = UpsampleBlock(base_filters * 4, 3)
        self.up2 = UpsampleBlock(base_filters * 2, 3)
        self.up3 = UpsampleBlock(base_filters, 3)

        self.final_norm = tf.keras.layers.GroupNormalization(axis=-1)
        self.final_conv = tf.keras.layers.Conv2D(3, 3, padding='same')

    def call(self, x, timesteps):
        t = self.time_embedding(timesteps)

        x = self.init_conv(x)
        x = self.first_resnet(x)
        skip1 = x

        x = self.down1([x, t])
        skip2 = x

        x = self.down2([x, t])
        skip3 = x

        x = self.down3([x, t])
        x = self.mid_attention(x)
        x = self.mid_resnet1(x)

        x = self.up1([x, skip3, t])
        x = self.up2([x, skip2, t])
        x = self.up3([x, skip1, t])

        x = self.final_norm(x)
        x = self.final_conv(x)

        return x

In [ ]:
model = DiffusionModel()
batch_size = 4
img = tf.random.normal((batch_size, 32, 32, 3))
t = tf.random.uniform((batch_size,), maxval=1000, dtype=tf.int32)
prediction = model(img, t)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:1331: UserWarning: Layer 'down_sample_block_24' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''Exception encountered when calling ResNetBlock.call().

Using a symbolic `tf.Tensor` as a Python `bool` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using Au

Output shape: (4, 8, 8, 3)


In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0
train_images = tf.expand_dims(train_images, axis=-1)
test_images = tf.expand_dims(test_images, axis=-1)

model = DiffusionModel(img_size=28, base_filters=32, time_embedding_dim=128)

def loss_fn(real_images, predicted_images):
    loss = tf.reduce_mean(tf.square(tf.cast(real_images, tf.float32) - tf.cast(predicted_images, tf.float32)))
    return loss

optimizer = Adam(learning_rate=0.001)

num_epochs = 5
batch_size = 32

loss_vals = []

for epoch in tqdm(range(num_epochs), desc="Epochs"):
    for step in tqdm(range(0, len(train_images), batch_size), desc="Batches", leave=False):
        batch_images = train_images[step:step+batch_size]
        batch_timesteps = tf.random.uniform((batch_size,), maxval=1000, dtype=tf.int32)

        with tf.GradientTape() as tape:
            predicted_images = model(batch_images, batch_timesteps)
            predicted_images_resized = tf.image.resize(predicted_images, [28, 28])  # Resize spatial dimensions
            predicted_images_resized = tf.image.rgb_to_grayscale(predicted_images_resized)
            loss = loss_fn(batch_images, predicted_images_resized)
            loss_vals.append(tuple([epoch, loss]))

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}")

test_timesteps = tf.random.uniform((len(test_images),), maxval=1000, dtype=tf.int32)
test_predictions = model(test_images, test_timesteps)
test_loss = loss_fn(test_images, test_predictions)
print(f"Test Loss: {test_loss:.4f}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:1331: UserWarning: Layer 'down_sample_block_30' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''Exception encountered when calling ResNetBlock.call().

Using a symbolic `tf.Tensor` as a Python `bool` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using Au

Epoch 1/5, Loss: 0.0294
